# 8. 딥러닝을 이용한 투자 전략

* 1. 캔틀차트 이미지로 변환해 CNN 이미지 분석을 통한 주가 방향을 예측
* 2. LSTM 모델로 주가 데이터 학습 후 다음날 주가 예측
* 3. 오토인코더를 활용한 주가 수익률데이터와 패턴이 비슷한 데이터를 만드는 방법과 압축된 정보를 새로운 특성으로 활용하는방법 

## 8.1 CNN을 활용한 캔들차트 예측 분석

* 과거 특정기간 (5, 10 ,20일) 캔들차트를 학습해 다음날 주가 흐름이 올라갈지 내려갈지를 분류하는 이진 분류

### 8.1.1 데이터
1. 데이터소스로부터 레이블 데이터 만들기 (수정종가, 오름내림으로 활용)
2. 데이터 소스로부터 캔들차트(설명 변수 데이터)만들기 (수정종가데이터를 이용한 이미지 만들기)
3. 레이블에 따라 캔들차트 분리하기 (오름내림에 따라 만든이미지를 레이블에 따라 0과 1폴더로 옮김)
4. 학습을 위해 캔들차트 복사하기 (원본데이터 훼손 방지)

In [66]:
import subprocess
import os
import sys

formatters = {
    'RED': '\033[91m',
    'GREEN': '\033[92m',
    'END': '\033[0m',
}

symbol = "BBNI.JK"
windows_length = "20"
dimension = "50"

training_start_date = "2000-01-01"
training_end_date = "2016-12-31"

testing_start_date = "2017-01-01"
testing_end_date = "2018-06-14"

download_data = False
onlytesting = True
onlytraining = True
use_volume = False

try:
    if download_data:
        if onlytesting:
            # get data testing
            print('{RED}\nGet Testing Data{END}'.format(**formatters))
            subprocess.call(
                f'python utils/get_data.py -sd {testing_start_date} -ed {testing_end_date} -t {symbol} -s yahoo -p testing', shell=True)
            print('{GREEN}Get Testing Data Done\n{END}'.format(**formatters))
        if onlytraining:
            # get data testing
            print('{RED}\nGet Training Data{END}'.format(**formatters))
            subprocess.call(
                f'python utils/get_data.py -sd {training_start_date} -ed {training_end_date} -t {symbol} -s yahoo -p training', shell=True)
            print('{GREEN}Get Training Data Done\n{END}'.format(**formatters))
except Exception as identifier:
    print(identifier)

try:
    if onlytraining:
        # create label training
        print('{RED}\nCreate Label Training Data{END}'.format(**formatters))
        subprocess.call(f'python preproccess_binclass.py -m createLabel -l {windows_length} -i stockdatas/{symbol}_training.csv', shell=True)
        print('{GREEN}Create Label Training Data Done\n{END}!'.format(**formatters))
    if onlytesting:
        # create label testing
        print('{RED}\nCreate Label Testing Data{END}'.format(**formatters))
        subprocess.call(f'python preproccess_binclass.py -m createLabel -l {windows_length} -i stockdatas/{symbol}_testing.csv', shell=True)
        print('{GREEN}Create Label Testing Data Done\n{END}'.format(**formatters))
except Exception as identifier:
    print(identifier)

try:
    if onlytraining:
        # convert to candlestick chart training data
        print('{RED}\nConvert Training Data to Candlestik{END}'.format(**formatters))
        subprocess.call(
            f'python preproccess_binclass.py -m ohlc2cs -l {windows_length} -i stockdatas/{symbol}_training.csv -t training -d {dimension} -v {use_volume}', shell=True)
        print('{GREEN}Convert Training Data to Candlestik Done\n{END}'.format(
            **formatters))
    if onlytesting:
        # convert to candlestick chart testing data
        print('{RED}\nConvert Testing Data to Candlestik{END}'.format(**formatters))
        subprocess.call(
            f'python preproccess_binclass.py -m ohlc2cs -l {windows_length} -i stockdatas/{symbol}_testing.csv -t testing -d {dimension} -v {use_volume}', shell=True)
        print('{GREEN}Convert Testing Data to Candlestik Done\n{END}'.format(
            **formatters))
except Exception as identifier:
    print(identifier)

try:
    if onlytraining:
        # labelling data training
        print('{RED}\nLabelling Training Data{END}'.format(**formatters))
        print(f'python preproccess_binclass.py -m img2dt -i dataset/{windows_length}_{dimension}/{symbol}/training -lf {symbol}_training_label_{windows_length}.txt')
        subprocess.call(
            f'python preproccess_binclass.py -m img2dt -i dataset/{windows_length}_{dimension}/{symbol}/training -lf {symbol}_training_label_{windows_length}.txt', shell=True)
        print('{GREEN}Labelling Training Data Done\n{END}'.format(**formatters))
    if onlytesting:
        # labelling data testing
        print('{RED}\nLabelling Testing Data{END}'.format(**formatters))
        subprocess.call(
            f'python preproccess_binclass.py -m img2dt -i dataset/{windows_length}_{dimension}/{symbol}/testing -lf {symbol}_testing_label_{windows_length}.txt', shell=True)
        print('{GREEN}Labelling Testing Data Done\n{END}'.format(**formatters))
except Exception as identifier:
    print(identifier)



Create Label Training Data
Create Label Training Data Done
!

Create Label Testing Data
Create Label Testing Data Done


Convert Training Data to Candlestik
Convert Training Data to Candlestik Done


Convert Testing Data to Candlestik
Convert Testing Data to Candlestik Done


Labelling Training Data
python preproccess_binclass.py -m img2dt -i dataset/20_50/BBNI.JK/training -lf BBNI.JK_training_label_20.txt
Labelling Training Data Done


Labelling Testing Data
Labelling Testing Data Done



In [ ]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import glob
import argparse
import os
from shutil import copyfile, move
from pathlib import Path

# https://github.com/matplotlib/mpl_finance
from mpl_finance import candlestick2_ochl, volume_overlay


def isnan(value):
    try:
        import math
        return math.isnan(float(value))
    except:
        return False


def removeOutput(finput):
    if(Path(finput)).is_file():
        os.remove(finput)


def main():
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('-i', '--input',
                        help='a csv file of stock data', required=True)
    parser.add_argument('-l', '--seq_len',
                        help='num of sequence length', default=20)
    parser.add_argument('-lf', '--label_file',
                        help='a label_file')
    parser.add_argument('-d', '--dimension',
                        help='a dimension value', type=int, default=48)
    parser.add_argument('-t', '--dataset_type',
                        help='training or testing datasets')
    parser.add_argument('-m', '--mode',
                        help='mode of preprocessing data', required=True)
    parser.add_argument('-v', '--use_volume',
                        help='combine with volume.', default=False)
    args = parser.parse_args()
    if args.mode == 'ohlc2cs':
        ohlc2cs(args.input, args.seq_len, args.dataset_type,
                args.dimension, args.use_volume)
    if args.mode == 'createLabel':
        createLabel(args.input, args.seq_len)
    if args.mode == 'img2dt':
        image2dataset(args.input, args.label_file)
    if args.mode == 'countImg':
        countImage(args.input)


def image2dataset(input, label_file):
    
    label_dict = {}
    with open(label_file) as f:
        for line in f:
            (key, val) = line.split(',')
            label_dict[key] = val.rstrip()
    
    path = "{}/{}".format(os.getcwd(), input)

    for filename in os.listdir(path):
        # print(filename)
        # print(os.getcwd())
        if filename is not '':
            for k, v in label_dict.items():
                splitname = filename.split("_")
                f, e = os.path.splitext(filename)
                # print("[DEBUG] - {}".format(splitname))
                newname = "{}_{}".format(splitname[0], splitname[1])
                if newname == k:
                    # print("{} same with {} with v {}".format(filename, k, v))
                    new_name = "{}{}.png".format(v, f)

                    os.rename("{}/{}".format(path, filename),
                              "{}/{}".format(path, new_name))
                    break

    folders = ['1', '0']
    for folder in folders:
        if not os.path.exists("{}/classes/{}".format(path, folder)):
            os.makedirs("{}/classes/{}".format(path, folder))

    for filename in os.listdir(path):
        if filename is not '' and filename is not 'classes':
            # print(filename[:1])
            ### 여기에 for k,v in label_dict.items() 돌면서
            f, e = os.path.splitext(filename)
            if label_dict[f] == "1":
                move("{}/{}".format(path, filename),
                     "{}/classes/1/{}".format(path, filename))
            elif label_dict[f] == "0":
                move("{}/{}".format(path, filename),
                     "{}/classes/0/{}".format(path, filename))

    print('Done')


def createLabel(fname, seq_len):
    # python preprocess.py -m createLabel -l 20 -i stockdatas/EWT_training5.csv
    print("Creating label . . .")
    # remove existing label file
    filename = fname.split('/')
    # print("{} - {}".format(filename[0], filename[1][:-4]))
    removeOutput("{}_label_{}.txt".format(filename[1][:-4], seq_len))

    df = pd.read_csv(fname, parse_dates=True, index_col=0)
    df.fillna(0)

    df.reset_index(inplace=True)
    df['Date'] = df['Date'].map(mdates.date2num)
    for i in range(0, len(df)):
        c = df.ix[i:i + int(seq_len), :]

        starting = 0
        endvalue = 0
        label = ""
        
        if len(c) == int(seq_len)+1:  ## 여기서 안나왔겠지. 마지막 부분이. 3450 3449
            # starting = c["Close"].iloc[-2]
            starting = c["Open"].iloc[-1]
            endvalue = c["Close"].iloc[-1]
            # print(f'endvalue {endvalue} - starting {starting}')
            tmp_rtn = endvalue / starting -1
            if tmp_rtn > 0:
                label = 1
            else:
                label = 0

            with open("{}_label_{}.txt".format(filename[1][:-4], seq_len), 'a') as the_file:
                the_file.write("{}-{},{}".format(filename[1][:-4], i, label))
                the_file.write("\n")
    print("Create label finished.")


def countImage(input):
    num_file = sum([len(files) for r, d, files in os.walk(input)])
    num_dir = sum([len(d) for r, d, files in os.walk(input)])
    print("num of files : {}\nnum of dir : {}".format(num_file, num_dir))


def ohlc2cs(fname, seq_len, dataset_type, dimension, use_volume):
    # python preprocess.py -m ohlc2cs -l 20 -i stockdatas/EWT_testing.csv -t testing
    print("Converting ohlc to candlestick")
    symbol = fname.split('_')[0]
    symbol = symbol.split('/')[1]
    print(symbol)
    path = "{}".format(os.getcwd())
    # print(path)
    if not os.path.exists("{}/dataset/{}_{}/{}/{}".format(path, seq_len, dimension, symbol, dataset_type)):
        os.makedirs("{}/dataset/{}_{}/{}/{}".format(path,seq_len, dimension, symbol, dataset_type))

    df = pd.read_csv(fname, parse_dates=True, index_col=0)
    df.fillna(0)
    plt.style.use('dark_background')
    df.reset_index(inplace=True)
    df['Date'] = df['Date'].map(mdates.date2num)
    # for i in range(0, len(df)):
    for i in range(0, len(df)-int(seq_len)):
        # ohlc+volume
        c = df.ix[i:i + int(seq_len) - 1, :]
        if len(c) == int(seq_len):
            my_dpi = 96
            fig = plt.figure(figsize=(dimension / my_dpi,
                                      dimension / my_dpi), dpi=my_dpi)
            ax1 = fig.add_subplot(1, 1, 1)
            candlestick2_ochl(ax1, c['Open'], c['Close'], c['High'],c['Low'],
                              width=1,colorup='#77d879', colordown='#db3f3f')
            ax1.grid(False)
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.xaxis.set_visible(False)
            ax1.yaxis.set_visible(False)
            ax1.axis('off')

            # create the second axis for the volume bar-plot
            # Add a seconds axis for the volume overlay
            if use_volume:
                ax2 = ax1.twinx()
                # Plot the volume overlay
                bc = volume_overlay(ax2, c['Open'], c['Close'], c['Volume'],
                                    colorup='#77d879', colordown='#db3f3f', alpha=0.5, width=1)
                ax2.add_collection(bc)
                ax2.grid(False)
                ax2.set_xticklabels([])
                ax2.set_yticklabels([])
                ax2.xaxis.set_visible(False)
                ax2.yaxis.set_visible(False)
                ax2.axis('off')
            pngfile = 'dataset/{}_{}/{}/{}/{}-{}.png'.format(
                seq_len, dimension, symbol, dataset_type, fname[11:-4], i)
            fig.savefig(pngfile, pad_inches=0, transparent=False)
            plt.close(fig)

            # Alpha 채널 없애기 위한.
            from PIL import Image
            img = Image.open(pngfile)
            img = img.convert('RGB')
            img.save(pngfile)

        # normal length - end

    print("Converting olhc to candlestik finished.")


if __name__ == '__main__':
    main()


In [67]:
import os
from shutil import copyfile
import sys


def cre8outputdir(pathdir, targetdir):
    # create folder output
    if not os.path.exists("{}/{}".format(pathdir, targetdir)):
        os.mkdir("{}/{}".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/train".format(pathdir, targetdir)):
        os.mkdir("{}/{}/train".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/test".format(pathdir, targetdir)):
        os.mkdir("{}/{}/test".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/train/0".format(pathdir, targetdir)):
        os.mkdir("{}/{}/train/0".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/train/1".format(pathdir, targetdir)):
        os.mkdir("{}/{}/train/1".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/test/0".format(pathdir, targetdir)):
        os.mkdir("{}/{}/test/0".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/test/1".format(pathdir, targetdir)):
        os.mkdir("{}/{}/test/1".format(pathdir, targetdir))


pathdir = "dataset"
origindir = "20_50/BBNI.JK"
targetdir = "dataset_BBNIJK_20_50"

cre8outputdir(pathdir, targetdir)

counttest = 0
counttrain = 0
for root, dirs, files in os.walk("{}/{}".format(pathdir, origindir)):
    for file in files:


        tmp = root.replace('\\','/')
        tmp_label = tmp.split('/')[-1]

        if tmp_label == '0':
            if 'test' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/test/0/{}".format(
                    pathdir, targetdir, file)
                copyfile(origin, destination)
                counttest += 1
            elif 'train' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/train/0/{}".format(
                    pathdir, targetdir, file)
                copyfile(origin, destination)
                counttrain += 1
        elif tmp_label == '1':
            if 'test' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/test/1/{}".format(
                    pathdir, targetdir, file)
                copyfile(origin, destination)
                counttest += 1
            elif 'train' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/train/1/{}".format(
                    pathdir, targetdir, file)
                copyfile(origin, destination)
                counttrain += 1

print(counttest)
print(counttrain)

348
3138


### 8.1.2 모델구조

* 그림 8-5

### 8.1.3 모델 학습

### 8.1.4 모델을 활용한 결과 예측

In [80]:
import tensorflow as tf
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

import json
import sys

#import keras
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Flatten, Activation, add
from tensorflow.keras.layers import Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import initializers
from keras.engine import Layer, InputSpec
from keras import backend as K
from keras.utils import np_utils
from tensorflow.keras.optimizers import *
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# from utils.dataset import dataset as dataset
from utils.dataset import dataset
import argparse

import time
from datetime import timedelta


def build_dataset(data_directory, img_width):
    # X, y, tags = dataset.dataset(data_directory, int(img_width))
    X, y, tags = dataset(data_directory, int(img_width))
    print(len(tags))
    nb_classes = len(tags)

    sample_count = len(y)
    train_size = sample_count
    print("train size : {}".format(train_size))
    feature = X
    label = np_utils.to_categorical(y, nb_classes)
    return feature, label, nb_classes

########################################################################
# 8.1.2 신경망 부분

def build_model(SHAPE, nb_classes, bn_axis, seed=None):

    if seed:
        np.random.seed(seed)

    input_layer = Input(shape=SHAPE)

    # Step 1 - 합성곱 32 Relu
    x = Conv2D(32, 3, 3, init='glorot_uniform',
               border_mode='same', activation='relu')(input_layer)
    # Step 2 - Pooling (최대풀링)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Step 1 - 합성곱 48 Relu
    x = Conv2D(48, 3, 3, init='glorot_uniform', border_mode='same',
               activation='relu')(x)
    # Step 2 - Pooling (최대풀링) 및 드롭아웃
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Step 1 - 합성곱 64 Relu
    x = Conv2D(64, 3, 3, init='glorot_uniform', border_mode='same',
               activation='relu')(x)
    # Step 2 - Pooling (최대풀링)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Step 1 - 합성곱 96 Relu
    x = Conv2D(96, 3, 3, init='glorot_uniform', border_mode='same',
               activation='relu')(x)
    # Step 2 - Pooling (최대풀링) 및 드롭아웃
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Step 3 - Flattening
    x = Flatten()(x)

    # Step 4 - Full connection
    # 밀집층 256
    x = Dense(output_dim=256, activation='relu')(x)
    # Dropout 드롭아웃
    x = Dropout(0.5)(x)
    # 밀집층 2
    x = Dense(output_dim=2, activation='softmax')(x)

    model = Model(input_layer, x)

    return model

def main():
    start_time = time.monotonic()
    # dimensions of our images.
    img_width, img_height = 50, 50
    channel = 3
    epochs = 50
    batch_size = 8
    SHAPE = (img_width, img_height, channel)
    bn_axis = 3 if K.image_dim_ordering() == 'tf' else 1

    # channel = 4
    # bn_axis = 4 if K.image_dim_ordering() == 'tf' else 1

    data_directory = "dataset/dataset_BBNIJK_20_50"

    print("loading dataset")
    X_train, Y_train, nb_classes = build_dataset(
        "{}/train".format(data_directory), 50) # dimension
    X_test, Y_test, nb_classes = build_dataset(
        "{}/test".format(data_directory), 50)  # dimension
    print("number of classes : {}".format(nb_classes))
    
    #########################################################################
    # 8.1.2 모델 구조
    model = build_model(SHAPE, nb_classes, bn_axis)
    
    
    #########################################################################
    # 8.1.3 모델학습
    model.compile(optimizer=Adam(lr=1.0e-4),
                  loss='categorical_crossentropy', metrics=['accuracy'])

    # Fit the model
    model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs)

    # Save Model or creates a HDF5 file
    model.save('{}epochs_{}batch_cnn_model_{}.h5'.format(
        epochs, batch_size, data_directory.replace("/", "_")), overwrite=True)
    
    ##########################################################################
    # 8.1.4 모델을 활용한 결과 예측
    # del model  # deletes the existing model
    predicted = model.predict(X_test)
    y_pred = np.argmax(predicted, axis=1)
    Y_test = np.argmax(Y_test, axis=1)
    cm = confusion_matrix(Y_test, y_pred)
    report = classification_report(Y_test, y_pred)
    tn = cm[0][0]
    fn = cm[1][0]
    tp = cm[1][1]
    fp = cm[0][1]
    if tp == 0:
        tp = 1
    if tn == 0:
        tn = 1
    if fp == 0:
        fp = 1
    if fn == 0:
        fn = 1
    TPR = float(tp)/(float(tp)+float(fn))
    FPR = float(fp)/(float(fp)+float(tn))
    accuracy = round((float(tp) + float(tn))/(float(tp) +
                                              float(fp) + float(fn) + float(tn)), 3)
    specitivity = round(float(tn)/(float(tn) + float(fp)), 3)
    sensitivity = round(float(tp)/(float(tp) + float(fn)), 3)
    mcc = round((float(tp)*float(tn) - float(fp)*float(fn))/math.sqrt(
        (float(tp)+float(fp))
        * (float(tp)+float(fn))
        * (float(tn)+float(fp))
        * (float(tn)+float(fn))
    ), 3)

    print('=======\n')
    print('{}epochs_{}batch_cnn\n'.format(epochs, batch_size))
    print('TN: {}\n'.format(tn))
    print('FN: {}\n'.format(fn))
    print('TP: {}\n'.format(tp))
    print('FP: {}\n'.format(fp))
    print('TPR: {}\n'.format(TPR))
    print('FPR: {}\n'.format(FPR))
    print('accuracy: {}\n'.format(accuracy))
    print('specitivity: {}\n'.format(specitivity))
    print("sensitivity : {}\n".format(sensitivity))
    print("mcc : {}\n".format(mcc))
    print("{}".format(report))
    print('=======\n')
    end_time = time.monotonic()
    print("Duration : {}".format(timedelta(seconds=end_time - start_time)))

    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    # Plot a confusion matrix.
    # cm is the confusion matrix, names are the names of the classes.
    def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(names))
        plt.xticks(tick_marks, names, rotation=45)
        plt.yticks(tick_marks, names)
        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

    # Plot an ROC. pred - the predictions, y - the expected output.
    def plot_roc(pred, y):
        fpr, tpr, _ = roc_curve(y, pred)
        roc_auc = auc(fpr, tpr)

        plt.figure()
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC)')
        plt.legend(loc="lower right")
        plt.savefig('ROC AUC.png')
        plt.show()


    plot_roc(y_pred, Y_test)


if __name__ == "__main__":
    main()


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

loading dataset
base_dir : dataset/dataset_BBNIJK_20_50/train, n : 50
root : dataset/dataset_BBNIJK_20_50/train
subdirs : ['0', '1']
files : []
root : dataset/dataset_BBNIJK_20_50/train\0
subdirs : []
files : ['BBNI.JK_training-0.png', 'BBNI.JK_training-1.png', 'BBNI.JK_training-10.png', 'BBNI.JK_training-100.png', 'BBNI.JK_training-1000.png', 'BBNI.JK_training-1001.png', 'BBNI.JK_training-1002.png', 'BBNI.JK_training-1004.png', 'BBNI.JK_training-1005.png', 'BBNI.JK_training-1008.png', 'BBNI.JK_training-1010.png', 'BBNI.JK_training-1011.png', 'BBNI.JK_training-1012.png', 'BBNI.JK_training-1013.png', 'BBNI.JK_training-1015.png', 'BBNI.JK_training-1016.png', 'BBNI.JK_training-1017.png', 'BBNI.JK_training-1019.png', 'BBNI.JK_training-1020.png', 'BBNI.JK_training-1021.png', 'BBNI.JK_training-1023.png', 'BBNI.JK_training-1024.png', 'BBNI.JK_training-1025.png', 'BBNI.JK_training-1026.png', 'BBNI.JK_tr

TypeError: ('Keyword argument not understood:', 'init')

In [ ]:
=======
50epochs_8batch_cnn
TN: 125
FN: 83
TP: 92
FP: 116
TPR: 0.5257142857142857
FPR: 0.48132780082987553
accuracy: 0.522
specitivity: 0.519
sensitivity : 0.526
mcc : 0.044
              precision    recall  f1-score   support

           0       0.60      0.52      0.56       241
           1       0.44      0.53      0.48       175

    accuracy                           0.52       416
   macro avg       0.52      0.52      0.52       416
weighted avg       0.53      0.52      0.52       416
=======
=======
50epochs_8batch_cnn
TN: 185
FN: 136
TP: 39
FP: 56
TPR: 0.22285714285714286
FPR: 0.23236514522821577
accuracy: 0.538
specitivity: 0.768
sensitivity : 0.223
mcc : -0.011
              precision    recall  f1-score   support

           0       0.58      0.77      0.66       241
           1       0.41      0.22      0.29       175

    accuracy                           0.54       416
   macro avg       0.49      0.50      0.47       416
weighted avg       0.51      0.54      0.50       416
=======
=======
50epochs_8batch_cnn
TN: 153
FN: 107
TP: 68
FP: 88
TPR: 0.38857142857142857
FPR: 0.3651452282157676
accuracy: 0.531
specitivity: 0.635
sensitivity : 0.389
mcc : 0.024
              precision    recall  f1-score   support

           0       0.59      0.63      0.61       241
           1       0.44      0.39      0.41       175

    accuracy                           0.53       416
   macro avg       0.51      0.51      0.51       416
weighted avg       0.52      0.53      0.53       416
=======
=======
50epochs_8batch_cnn
TN: 127
FN: 87
TP: 88
FP: 114
TPR: 0.5028571428571429
FPR: 0.4730290456431535
accuracy: 0.517
specitivity: 0.527
sensitivity : 0.503
mcc : 0.029
              precision    recall  f1-score   support

           0       0.59      0.53      0.56       241
           1       0.44      0.50      0.47       175

    accuracy                           0.52       416
   macro avg       0.51      0.51      0.51       416
weighted avg       0.53      0.52      0.52       416
=======
=======
50epochs_8batch_cnn
TN: 165
FN: 112
TP: 63
FP: 76
TPR: 0.36
FPR: 0.3153526970954357
accuracy: 0.548
specitivity: 0.685
sensitivity : 0.36
mcc : 0.047
              precision    recall  f1-score   support

           0       0.60      0.68      0.64       241
           1       0.45      0.36      0.40       175

    accuracy                           0.55       416
   macro avg       0.52      0.52      0.52       416
weighted avg       0.54      0.55      0.54       416
=======
=======
50epochs_8batch_cnn
TN: 178
FN: 118
TP: 57
FP: 63
TPR: 0.32571428571428573
FPR: 0.26141078838174275
accuracy: 0.565
specitivity: 0.739
sensitivity : 0.326
mcc : 0.07
              precision    recall  f1-score   support

           0       0.60      0.74      0.66       241
           1       0.47      0.33      0.39       175

    accuracy                           0.56       416
   macro avg       0.54      0.53      0.52       416
weighted avg       0.55      0.56      0.55       416
=======

In [72]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.__version__)

2.3.1
2.4.0


In [52]:
tf.keras.__version__

'2.4.0'

In [49]:
import keras

In [56]:
conda install -n test tensorflow-gpu


Note: you may need to restart the kernel to use updated packages.



EnvironmentLocationNotFound: Not a conda environment: C:\Users\user\anaconda3\envs\test



In [55]:
!pip uninstall --force tensorflow


Usage:   
  pip uninstall [options] <package> ...
  pip uninstall [options] -r <requirements file> ...

no such option: --force


In [62]:
tf.compat.v1.get_default_graph()